In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

from pymedphys.dicom import depth_dose, profile

In [ ]:
algorithm = 'MC'
energy = '10FFF'
test_field = 'TandG_Odd'

filename = algorithm + energy + '_' + test_field

filename

In [ ]:
if algorithm == 'MC':
    algorithm_full = 'Monte Carlo'
    
if algorithm == 'CC':
    algorithm_full = 'Collapsed Cone'
    
algorithm_full

In [ ]:
ROOT_DIR = Path(r"S:\Physics\DoseCHECK\Commissioning\MLC test field Results\DICOM Comparisons")

MONACO_DICOM_DIR = (ROOT_DIR.joinpath(r"DICOM Exports\Monaco")).joinpath(filename)
DOSECHECK_DICOM_DIR = (ROOT_DIR.joinpath(r"DICOM Exports\DoseCHECK")).joinpath(filename)
RESULTS = ROOT_DIR.joinpath(r"Results")

MONACO_DICOM_DIR

In [ ]:
FILM_DIR = Path(r"S:\Physics\RCCC Specific Files\Dosimetry\Film\TandG\20190904")

FILM_RESULTS_FILE = FILM_DIR.joinpath(f"{energy + '_' + test_field}.csv")

film_results = pd.read_csv(FILM_RESULTS_FILE)

In [ ]:
film_dose = film_results.Dose.values

In [ ]:
monaco_dose = pydicom.read_file(str(MONACO_DICOM_DIR.joinpath('dose.dcm')), force=True)
dosecheck_dose = pydicom.read_file(str(DOSECHECK_DICOM_DIR.joinpath('dose.dcm')), force=True)
dicom_plan = pydicom.read_file(str(MONACO_DICOM_DIR.joinpath('plan.dcm')), force=True)

In [ ]:
start_cache = {
    '06FFF': -127,
    '06MV': -125,
    '10FFF': -123,
    '10MV': -124
}

start = start_cache[energy]
step = 0.1
end = start + step * len(film_dose)

plane_dimensions = np.arange(start,end,step)

len(plane_dimensions)

In [ ]:
monaco_inplane_profile = profile(plane_dimensions,20,'inplane',monaco_dose,dicom_plan)

dosecheck_inplane_profile = profile(plane_dimensions,20,'inplane',dosecheck_dose,dicom_plan)

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(plane_dimensions,monaco_inplane_profile,label='Monaco')
plt.plot(plane_dimensions,dosecheck_inplane_profile,label='DoseCHECK')
plt.plot(plane_dimensions,film_dose[::-1]/100,label='Film')
plt.ylabel('Dose (Gy / 100 MU)')
plt.xlabel('Plane Position (mm)')
plt.title('Inplane Profiles at Depth of 20 mm')
plt.legend(loc='lower left')


plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)

plt.suptitle(('MLC Test Fields, ' + energy + ' ' + test_field + ' , Doses for Monaco ' + algorithm_full + ', DoseCHECK, and film'),fontsize="x-large",x=1,y=3.2)

plt.savefig(RESULTS.joinpath(filename + f'_1.png'),bbox_inches='tight')

In [ ]:
[100 * np.min(monaco_inplane_profile),100 * np.min(dosecheck_inplane_profile)]

In [ ]:
[100 * np.max(monaco_inplane_profile),100 * np.max(dosecheck_inplane_profile)]